In [3]:
import dask.array as da
import dask
import zarr
import cv2
import numpy as np
import matplotlib

In [4]:
from MotionEnergyAnalyzer import MotionEnergyAnalyzer
import utils

In [5]:
# notebook dev
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:

zarr_paths = utils.find_zarr_paths(subselect='Face')

In [10]:
zarr_paths

[]

In [7]:
analyser = MotionEnergyAnalyzer(zarr_paths[0])

IndexError: list index out of range

In [7]:
analyser.analyze()

<class 'dask.array.core.Array'>
<class 'dask.array.core.Array'>
Video saved to '/root/capsule/results/721291_Face_21707174-ce69-4ce2-b7b5-9e0ce26bbc59/motion_energy_clip.avi'
/root/capsule/results/721291_Face_21707174-ce69-4ce2-b7b5-9e0ce26bbc59/motion_energy_frames.zarr
Saved motion energy frames to /root/capsule/results/721291_Face_21707174-ce69-4ce2-b7b5-9e0ce26bbc59/motion_energy_frames.zarr


In [10]:
len(analyser.motion_energy_sum)

272033

In [12]:
len(analyser.loaded_metadata.get('timestamps'))

271980

In [ ]:

# Load the grayscale frames from the Zarr file
zarr_store_frames = zarr.DirectoryStore('video_frames_grayscale.zarr')
grayscale_frames = da.from_zarr(zarr_store_frames, component='frames')

# Compute motion energy as the absolute difference between consecutive frames
# This will be a Dask array operation that only loads and computes chunks as needed
motion_energy = da.abs(grayscale_frames[1:] - grayscale_frames[:-1])

# Define the chunking for efficient memory usage; rechunk if necessary
motion_energy = motion_energy.rechunk((100, 658, 492))  # Adjust based on available memory

# Save motion energy as a new Zarr file
zarr_store_motion = zarr.DirectoryStore('motion_energy_grayscale.zarr')
motion_energy.to_zarr(zarr_store_motion, overwrite=True)

# Compute the sum of motion energy for each frame and save as a NumPy array
# This sums across height and width (658, 492) to get the sum per frame (along the time dimension)
motion_energy_sum = motion_energy.sum(axis=(1, 2)).compute()

# Save the result as a NumPy array
np.save('motion_energy_sum.npy', motion_energy_sum)